In [ ]:
import re

input_file = "inference.txt"
output_file = "1.txt"


scores = []


with open(input_file, "r", encoding="utf-8") as file:
    lines = file.readlines()

for line in lines:
    
    match = re.search(r'Output:\s*(.*?)\s*Explanation', line)
    if match:
        score = match.group(1).strip()
        scores.append(score)
    else:
        scores.append("")  


with open(output_file, "w", encoding="utf-8") as file:
    file.write("\n".join(scores))

print(f"✅  completed, results saved to {output_file}")

In [ ]:

input_file = "1.txt"
output_file = "filled_1.txt"


with open(input_file, "r", encoding="utf-8") as file:
    lines = file.readlines()


processed_lines = ["0" if line.strip() == "" else line.strip() for line in lines]


with open(output_file, "w", encoding="utf-8") as file:
    file.write("\n".join(processed_lines) + "\n") 

print(f"✅ Processing completed, results saved to {output_file}")

In [ ]:
import re


input_file = "filled_1.txt"
output_file = "processed_filled_1.txt"


with open(input_file, "r", encoding="utf-8") as file:
    lines = file.readlines()


replacements = {
    "```3;```###": "3",
    "1```###": "1",
    "```4```###": "4",
    "5###": "5",
    "```4;```###": "4",
    "4###": "4",
    "4```###": "4",
    "```5;```###": "5",
    "5```###": "5",
    "```4```###": "4",
    "```5```###": "5",
    "444Sample": "4",
    "4.0": "4",
    "4Sample": "4",
    "10.00.00.0": "0",
    "1111": "1",
    "1 1 1 1 1 1 1 1": "1",
    "1 4 4 4 4": "4",
    "1 4 4 4 41 4 4 4 41 4 4 4 4": "4",
    "1 4 4 4 4 4 41 4 4 4 4 4 41 4 4 4 4 4 4": "4",
    "1 1 1 1 1 1 1": "1",
    "1 1 1 1 1 1": "1",
    "1 1 1 1 1": "1",
    "1 4 3 5": "3",
    "10.020.030.0": "0",
    "10.010.010.0": "0",
    "123": "2",
    "1 1 1 1 1 1 1 1 1 1": "1",
    "```0.0```###": "0",
    "```1```###": "1",
    "110.0": "0",
    "5.05.05.0": "5",
    "5.0": "5",
    "0.0": "0",
    "111": "1",
    "0.00.00.0": "0",
    "4.04.04.0": "4",
    "444": "4",
    "1.01.01.0": "1",
    "555": "5",
    "0.0###": "0",
    "4.0###": "4",
    "5.0###": "5"
}


processed_lines = []
for line in lines:
    line = line.strip() 
    for pattern, replacement in replacements.items():
        if re.fullmatch(pattern, line): 
            line = replacement
            break  
    processed_lines.append(line)


with open(output_file, "w", encoding="utf-8") as file:
    file.write("\n".join(processed_lines) + "\n")  

print(f"✅ Processing completed, results saved to {output_file}")

In [ ]:
import re
import json
import csv


jsonl_file = "cleaned.jsonl"
scores_file = "processed_filled_1.txt"
output_csv = "movies_with_scores.csv"


with open(scores_file, "r", encoding="utf-8") as file:
    scores = [line.strip() for line in file.readlines()]


movies = []
with open(jsonl_file, "r", encoding="utf-8") as file:
    for line in file:
        data = json.loads(line)
        inst = data.get("inst", "")

        
        match = re.search(r'the target music\s*"(.*?)"\s*with the user', inst)
        if match:
            movie_name = match.group(1)
        else:
            movie_name = "Unknown"

        movies.append(movie_name)


if len(movies) != len(scores):
    print("⚠️ Warning: Number of movies does not match number of ratings！")
else:
    print("✅ Data matching is successful, start writingCSV...")

# 写入 CSV 文件
with open(output_csv, "w", encoding="utf-8", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Movie", "Score"])  # 写入表头
    for movie, score in zip(movies, scores):
        writer.writerow([movie, score])

print(f"✅ Processing completed, results saved to {output_csv}")


In [ ]:
import pandas as pd


input_csv = "movies_with_scores.csv"
output_csv = "movies_recommendations.csv"


df = pd.read_csv(input_csv)


movies = df.iloc[:, 0].tolist()  
scores = df.iloc[:, 1].astype(float).tolist()  


alpha1 = 0.05


num_movies = len(movies)
group_size = 10  
recommendations = []

for i in range(0, num_movies, group_size):
    movie_group = movies[i:i+group_size] 
    score_group = scores[i:i+group_size]  

    
    if len(movie_group) < group_size:
        continue

   
    adjusted_scores = [score + alpha1 * (group_size - rank) for rank, score in enumerate(score_group)]

    
    sorted_movies = [movie for _, movie in sorted(zip(adjusted_scores, movie_group), reverse=True)]

   
    recommendations.append([len(recommendations) + 1] + sorted_movies[:5])


columns = ["Line Number", "Movie 1", "Movie 2", "Movie 3", "Movie 4", "Movie 5"]
df_output = pd.DataFrame(recommendations, columns=columns)
df_output.to_csv(output_csv, index=False)

print(f"✅ Processing completed, results saved to {output_csv}")


In [7]:
import pandas as pd

def merge_files(user_item_file, movie_info_file, output_file):
    
    user_item_df = pd.read_csv(user_item_file, header=None, names=['user_id', 'item_id'])
    
    
    movie_info_df = pd.read_csv(movie_info_file, sep='|', header=None, names=['item_id', 'movie_name'],engine='python', encoding='latin-1')

    
    
    merged_df = pd.merge(user_item_df, movie_info_df[['item_id', 'movie_name']], on='item_id', how='left')
    
    
    merged_df.insert(0, 'index', range(1, len(merged_df) + 1))
    
   
    final_df = merged_df[['index', 'item_id', 'movie_name']]
    
    
    final_df.to_csv(output_file, index=False, encoding='utf-8')

# Usage
merge_files('XSimGCLgt_save_dict1.csv', 'itemnew-processed.csv', 'gt-match-output.csv')


In [8]:
import pandas as pd

# Load the two CSV files
movies_recommendations = pd.read_csv('movies_recommendations.csv')
match_output = pd.read_csv('gt-match-output.csv')

# Merge the files based on the 'Line Number' from movies_recommendations and 'index' from match_output
merged_df = pd.merge(movies_recommendations, match_output[['index', 'movie_name']], how='left', left_on='Line Number', right_on='index')

# Drop the extra 'index' column after the merge
merged_df = merged_df.drop(columns=['index'])

# Save the merged result to a new CSV file
output_file = 'merged_with_movie_name.csv'
merged_df.to_csv(output_file, index=False)

print(f"CSV file created at: {output_file}")

CSV file created at: merged_with_movie_name.csv


In [9]:
import pandas as pd

# Load the merged CSV file
merged_df = pd.read_csv('merged_with_movie_name.csv')

# Function to check if the movie_name matches any of Movie1 to Movie5
def check_match(row):
    for i in range(1, 6):
        if row['movie_name'] == row[f'Movie {i}']:
            return i
    return 0

# Apply the check_match function to each row and create a new column for the result
merged_df['Match'] = merged_df.apply(check_match, axis=1)

# Save the updated dataframe with the new column
output_file_with_match = 'merged_with_match_column.csv'
merged_df.to_csv(output_file_with_match, index=False)

print(f"CSV file created at: {output_file_with_match}")

CSV file created at: merged_with_match_column.csv


In [ ]:
import pandas as pd
import numpy as np

# Load the merged CSV file with the match column
df = pd.read_csv('merged_with_match_column.csv')

# Function to calculate Hit@n
def calculate_hit(df, n):
    return df['Match'].apply(lambda x: 1 if 0 < x <= n else 0).mean()

# Function to calculate DCG@n and NDCG@n for each row
def calculate_dcg(row, n):
    match = row['Match']
    if 0 < match <= n:  # if there's a match in the top n
        return 1 / np.log2(match + 1)
    return 0

# Function to calculate NDCG@n
def calculate_ndcg(df, n):
    # Calculate DCG@n for each row
    df[f'DCG@{n}'] = df.apply(lambda row: calculate_dcg(row, n), axis=1)
    # Ideal DCG@n (IDCG@n) is always 1, since the best match is in the first position
    idcg = 1 / np.log2(2)
    # Calculate NDCG@n for each row
    df[f'NDCG@{n}'] = df[f'DCG@{n}'] / idcg
    return df[f'NDCG@{n}'].mean()

# Calculate Hit@3 and NDCG@3
hit_at_3 = calculate_hit(df, 3)
ndcg_at_3 = calculate_ndcg(df, 3)

# Calculate Hit@5 and NDCG@5
hit_at_5 = calculate_hit(df, 5)
ndcg_at_5 = calculate_ndcg(df, 5)

# Scale factor508 / 6035   127 / 942  103 / 1162
scale_factor = 120/ 1162

# Scaled results
scaled_hit_at_3 = hit_at_3 * scale_factor
scaled_ndcg_at_3 = ndcg_at_3 * scale_factor
scaled_hit_at_5 = hit_at_5 * scale_factor
scaled_ndcg_at_5 = ndcg_at_5 * scale_factor

# Output the results
print(f"Hit@3: {hit_at_3}")
print(f"NDCG@3: {ndcg_at_3}")
print(f"Hit@5: {hit_at_5}")
print(f"NDCG@5: {ndcg_at_5}")

print("\n### Scaled Results (Multiplied by 120/ 1162) ###")
print(f"Scaled Hit@3: {scaled_hit_at_3}")
print(f"Scaled NDCG@3: {scaled_ndcg_at_3}")
print(f"Scaled Hit@5: {scaled_hit_at_5}")
print(f"Scaled NDCG@5: {scaled_ndcg_at_5}")